In [4]:
import numpy as np
from ase.io import read, write
from ase import Atoms
from dpyscfl.utils import ase_atoms_to_mol, gen_mf_mol
#for pylibnxc version
import torch, pylibnxc
from pyscf import dft
#for dpyscfl
from dpyscfl.net import get_scf
from dpyscfl.utils import old_get_datapoint
#for original+xcdiff dpyscf
from dpyscf.utils import get_datapoint
from dpyscf.net import lite_get_scf

In [5]:
import dpyscf
dpyscf.__file__

'/home/awills/Documents/Research/xcdiff/dpyscf/__init__.py'

# Case study: O3+O

In [4]:
# training dataset, O3 is trj[24], O is 20
trj = read('/home/awills/Documents/Research/sebshare/data/09/subat_ref.traj', ':')
g2o3 = trj[24]
print(g2o3.positions)
print(g2o3.info)
g2o = trj[20]
print(g2o.positions)
print(g2o.info)

[[ 0.        1.075722 -0.21567 ]
 [ 0.        0.        0.43134 ]
 [ 0.       -1.075722 -0.21567 ]]
{'O3': True, 'name': 'Ozone', 'openshell': False, 'sc': False, 'sym': False, 'n_rad': 1, 'grid_level': 1, 'pol': False, 'target_energy': -0.2345382154735096, 'energy': -225.20206708569773, 'atomization': -0.2345382154735096, 'atomization_ev': -6.382109918715892, 'atomization_H': -0.2345382154735096, 'calc_energy': -225.20206708569773, 'e_calc': -225.20206708569773}
[[0. 0. 0.]]
{'spin': 2, 'sym': False, 'grid_level': 4, 'basis': '6-311++G(3df,2pd)', 'pol': False, 'target_energy': -75.0673, 'energy': -74.99416120454062, 'atomization': -75.0673, 'atomization_ev': -2042.6852781069833, 'atomization_H': -75.0673, 'calc_energy': -75.0673, 'e_calc': -75.0673}


In [6]:
#positions from training dataset, but recreation to show how one makes the Atoms object
o3 = Atoms(['O','O','O'], positions = [[ 0.,        1.075722, -0.21567 ], 
                                       [ 0.,        0.,        0.43134 ],
                                       [ 0.,       -1.075722, -0.21567 ]])
o3.info = {'O3': True, 
           'name': 'Ozone', 
           'openshell': False, 
           'sc': False, 
           'sym': False, 
           'n_rad': 1, 
           'grid_level': 1, 
           'pol': False, 
           'target_energy': -0.2345382154735096, 
           'energy': -225.20206708569773, 
           'atomization': -0.2345382154735096, 
           'atomization_ev': -6.382109918715892, 
           'atomization_H': -0.2345382154735096, 
           'calc_energy': -225.20206708569773, 
           'e_calc': -225.20206708569773}
o = Atoms(['O'], positions=[[0,0,0]])
o.info = {'spin': 2, 
          'sym': False, 
          'grid_level': 4, 
          'basis': '6-311++G(3df,2pd)', 
          'pol': False, 
          'target_energy': -75.0673, 
          'energy': -74.99416120454062, 
          'atomization': -75.0673, 
          'atomization_ev': -2042.6852781069833, 
          'atomization_H': -75.0673, 
          'calc_energy': -75.0673, 
          'e_calc': -75.0673}

#openshell is false, 0 spin ozone -- generate the pyscf Mol objects to be used below
name_o3, mol_o3 = ase_atoms_to_mol(o3, basis='def2-tzvp', charge=0, spin=0)
name_o, mol_o = ase_atoms_to_mol(o, basis='def2-tzvp', charge=0, spin=2)

# PySCF

In [3]:
#generate the mf and method from the mol and options
mf1_o3, method1_o3 = gen_mf_mol(mol_o3, xc='scan', pol=True, grid_level=3, nxc = False)
#run, save energy
mf1_o3.kernel()
e1_o3 = mf1_o3.e_tot

mf1_o, method1_o = gen_mf_mol(mol_o, xc='scan', pol=True, grid_level=3, nxc = False)
mf1_o.kernel()
e1_o = mf1_o.e_tot

print('PySCF (SCAN) Evaluated on:\nO3 = {}\nO = {}'.format(e1_o3, e1_o))

Building grids...
METHOD GENERATED: <function UKS at 0x7fdded2bf170>
converged SCF energy = -225.440612971224  <S^2> = 1.2447288e-10  2S+1 = 1
Single Atom -- UKS
Building grids...
METHOD GENERATED: <function UKS at 0x7fdded2bf170>
SCF not converged.
SCF energy = -75.0702071065462 after 50 cycles  <S^2> = 2.0048584  2S+1 = 3.0032372
PySCF (SCAN) Evaluated on:
O3 = -225.44061297122408
O = -75.07020710654618


# Pylibnxc

In [4]:
def KS(mol, method, model_path='', nxc_kind='grid', **kwargs):
    """ Wrapper for the pyscf RKS and UKS class
    that uses a libnxc functionals
    """
    #hyb = kwargs.get('hyb', 0)
    mf = method(mol, **kwargs)
    if model_path != '':
        if nxc_kind.lower() == 'atomic':
            model = get_nxc_adapter('pyscf', model_path)
            mf.get_veff = veff_mod_atomic(mf, model)
        elif nxc_kind.lower() == 'grid':
            parsed_xc = pylibnxc.pyscf.utils.parse_xc_code(model_path)
            dft.libxc.define_xc_(mf._numint,
                                 eval_xc,
                                 pylibnxc.pyscf.utils.find_max_level(parsed_xc),
                                 hyb=parsed_xc[0][0])
            mf.xc = model_path
        else:
            raise ValueError(
                "{} not a valid nxc_kind. Valid options are 'atomic' or 'grid'"
                .format(nxc_kind))
    return mf


def eval_xc(xc_code, rho, spin=0, relativity=0, deriv=1, verbose=None):
    """ Evaluation for grid-based models (not atomic)
        See pyscf documentation of eval_xc
    """
    inp = {}
    if spin == 0:
        if rho.ndim == 1:
            rho = rho.reshape(1, -1)
        inp['rho'] = rho[0]
        if len(rho) > 1:
            dx, dy, dz = rho[1:4]
            gamma = (dx**2 + dy**2 + dz**2)
            inp['sigma'] = gamma
        if len(rho) > 4:
            inp['lapl'] = rho[4]
            inp['tau'] = rho[5]
    else:
        rho_a, rho_b = rho
        if rho_a.ndim == 1:
            rho_a = rho_a.reshape(1, -1)
            rho_b = rho_b.reshape(1, -1)
        inp['rho'] = np.stack([rho_a[0], rho_b[0]])
        if len(rho_a) > 1:
            dxa, dya, dza = rho_a[1:4]
            dxb, dyb, dzb = rho_b[1:4]
            gamma_a = (dxa**2 + dya**2 + dza**2)  #compute contracted gradients
            gamma_b = (dxb**2 + dyb**2 + dzb**2)
            gamma_ab = (dxb * dxa + dyb * dya + dzb * dza)
            inp['sigma'] = np.stack([gamma_a, gamma_ab, gamma_b])
        if len(rho_a) > 4:
            inp['lapl'] = np.stack([rho_a[4], rho_b[4]])
            inp['tau'] = np.stack([rho_a[5], rho_b[5]])

    parsed_xc = pylibnxc.pyscf.utils.parse_xc_code(xc_code)
    total_output = {'v' + key: 0.0 for key in inp}
    total_output['zk'] = 0
    #print(parsed_xc)
    for code, factor in parsed_xc[1]:
        model = pylibnxc.LibNXCFunctional(xc_code, kind='grid')
        output = model.compute(inp)
        for key in output:
            if output[key] is not None:
                total_output[key] += output[key] * factor

    exc, vlapl, vtau, vrho, vsigma = [total_output.get(key,None)\
      for key in ['zk','vlapl','vtau','vrho','vsigma']]

    vxc = (vrho, vsigma, vlapl, vtau)
    fxc = None  # 2nd order functional derivative
    kxc = None  # 3rd order functional derivative
    return exc, vxc, fxc, kxc


In [5]:
#location of XCDiff, the network is stored within in file 'xc'
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA'
#this really just generates the method to pass into KS, since the mf is created in KS
_, method2_o3 = gen_mf_mol(mol_o3, xc='notnull', pol=True, grid_level=3, nxc=True)
#use wrapper to sent model to pylibnxc, generate and save energy
mf2_o3 = KS(mol_o3, method2_o3, model_path=modelpath)
mf2_o3.kernel()
e2_o3 = mf2_o3.e_tot

_, method2_o = gen_mf_mol(mol_o, xc='notnull', pol=True, grid_level=3, nxc=True)
mf2_o = KS(mol_o, method2_o, model_path=modelpath)
mf2_o.kernel()
e2_o = mf2_o.e_tot

print('Pylibnxc (XCDiff) Evaluated on:\nO3 = {}\nO = {}'.format(e2_o3, e2_o))

Building grids...
METHOD GENERATED: functools.partial(<function KS at 0x7fdded2d0680>, method=<function UKS at 0x7fdded2bf170>)


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)


converged SCF energy = -225.170147070068  <S^2> = 9.7912789e-12  2S+1 = 1
Single Atom -- UKS
Building grids...
METHOD GENERATED: functools.partial(<function KS at 0x7fdded2d0680>, method=<function UKS at 0x7fdded2bf170>)
converged SCF energy = -74.9819588240475  <S^2> = 2.0043007  2S+1 = 3.0028658
Pylibnxc (XCDiff) Evaluated on:
O3 = -225.1701470700678
O = -74.98195882404748


# Dpyscfl

**WARNING: On my machine at least, PyTorch *does not release the memory used during an SCF cycle.* Doing the above calculations with 25 SCF steps uses ~50 GB of RAM, which might not get released until restarting this notebook's kernel. User beware.**

In [6]:
#generate pytorch matrices based on mol and options
mats_o3 = old_get_datapoint(o3, basis='6-311++G(3df,2pd)', grid_level= 3,
                                xc='scan', zsym=o3.info.get('sym', False),
                                n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
                                init_guess=False, spin = 0,
                                pol=True, do_fcenter=True,
                                ref_path='/home/awills/Documents/Research/sebshare/data/09', ref_index= 24, ref_traj='subat_ref.traj',
                                ref_basis='6-311++G(3df,2pd)', dfit=False)
mats_o = old_get_datapoint(o, basis='6-311++G(3df,2pd)', grid_level= 3,
                                xc='scan', zsym=o.info.get('sym', False),
                                n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
                                init_guess=False, spin = 2,
                                pol=True, do_fcenter=True,
                                ref_path='/home/awills/Documents/Research/sebshare/data/09', ref_index= 20, ref_traj='subat_ref.traj',
                                ref_basis='6-311++G(3df,2pd)', dfit=False)
#unpack returned tuples
e_base_o3, eye_o3, mats_o3 = mats_o3
e_base_o, eye_o, mats_o = mats_o

Atoms(symbols='O3', pbc=False)
6-311++G(3df,2pd)
No specifications in atom info to help, assuming no spin.
Building grids...
METHOD GENERATED: <function UKS at 0x7fdded2bf170>
converged SCF energy = -225.426594202772  <S^2> = 6.1504579e-11  2S+1 = 1


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/pyscf/gto/moleintor.py:259: UserWarning: Function int4c1e_sph not found.  Set its comp to 1
  warnings.warn('Function %s not found.  Set its comp to 1' % intor_name)


DATAPOINT E_BASE:  -225.42659420277153
DATAPOINT MO_ENERGY:  [[-1.92571100e+01 -1.90762671e+01 -1.90762518e+01 -1.34036886e+00
  -1.07060292e+00 -7.53192353e-01 -5.75704506e-01 -5.67891687e-01
  -5.61150410e-01 -3.48594750e-01 -3.15168471e-01 -3.05517948e-01
  -2.25212138e-01  3.81674387e-02  9.40075120e-02  1.01851496e-01
   1.41693816e-01  1.50795435e-01  1.75940012e-01  2.18854400e-01
   2.19385694e-01  2.19898701e-01  2.34668605e-01  2.40682916e-01
   3.02788037e-01  3.27238741e-01  3.67685060e-01  6.13849116e-01
   6.99187710e-01  7.29281308e-01  8.00291718e-01  8.38083067e-01
   8.62844724e-01  8.75352963e-01  9.12076168e-01  9.18789589e-01
   9.20247624e-01  9.46069888e-01  9.59753268e-01  1.00309848e+00
   1.00708075e+00  1.01139283e+00  1.02698762e+00  1.06247400e+00
   1.12188642e+00  1.23539808e+00  1.39037006e+00  1.41856166e+00
   1.45392406e+00  1.53416391e+00  1.71562933e+00  1.79968041e+00
   2.20064072e+00  2.39232935e+00  2.83183254e+00  2.86748171e+00
   2.96687913e+

In [7]:
#need to generate more matrices for network -- this is usually handled by the data reader class used in training
s_chol_o3 = np.linalg.inv(np.linalg.cholesky(mats_o3['s']))
s_chol_o = np.linalg.inv(np.linalg.cholesky(mats_o['s']))

mats_o3['s_chol'] = s_chol_o3
mats_o['s_chol'] = s_chol_o

#go through and make everything a list, since everything in the scripts assumes matrices stored as 
# key: [mat_idx0, mat_idx1, ...]
mats_o3_l = {k: [mats_o3[k]] for k in mats_o3.keys()}
mats_o_l = {k: [mats_o[k]] for k in mats_o.keys()}

#go through and additionally make everything a tensor, since forward pass expects things to be tensors
for k in mats_o3_l.keys():
    print(k)
    o3k = mats_o3_l[k]
    ok = mats_o_l[k]
    if type(o3k[0]) in [int, float]:
        print('integer/float from o3 dict')
        mats_o3_l[k] = [torch.Tensor(np.array(o3k))]
        mats_o_l[k] = [torch.Tensor(np.array(ok))]
    elif not o3k[0].shape:
        mats_o3_l[k] = [torch.Tensor(o3k[0].reshape(1,))]
        mats_o_l[k] = [torch.Tensor(ok[0].reshape(1,))]
    else:
        mats_o3_l[k] = [torch.Tensor(o3k[0])]
        mats_o_l[k] = [torch.Tensor(ok[0])]

dm_init
dm
v
t
s
eri
n_elec
n_atoms
integer/float from o3 dict
e_nuc
mo_energy
mo_occ
[array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 

In [8]:
#location of model
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA'
#type MGGA specified, but additionally get_scf expects the full file, so add /xc to path
scf = get_scf('MGGA', path=modelpath+'/xc')
#do self-consistent
sc=True
#added recently -- prints brief info each pass
vvv = True
#generate results dictionary and then select energy only
res_o3 = scf([torch.Tensor(mats_o3_l['dm_init'][0])], mats_o3_l, sc=sc, verbose=vvv)
res_o3 = res_o3['E']

res_o = scf([torch.Tensor(mats_o_l['dm_init'][0])], mats_o_l, sc=sc, verbose=vvv)
res_o = res_o['E']

print('Dpyscf-lite (XCDiff) Evaluated on:\nO3 = {}\nO = {}'.format(res_o3, res_o))

FREEC False
SCF Loop Beginning: 25 Steps
Step 0
Noise generation to avoid potential degeneracies
0 Energy: tensor([-225.1551], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-225.1555], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-225.1556], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.15

17 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 18
18 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor(

Noise generation to avoid potential degeneracies
0 Energy: tensor([-74.9761], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-74.9764], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-74.9764], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackw

18 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<

The output of the above cell is pasted at the end of the cell in the event the generated output above gets erased.

# Dpyscf (original, contained in xcdiff folder)

**WARNING: On my machine at least, PyTorch *does not release the memory used during an SCF cycle.* Doing the above calculations with 25 SCF steps uses ~50 GB of RAM, which might not get released until restarting this notebook's kernel. User beware.**

In [6]:
ref_path = '/home/awills/Documents/Research/swxcd/aegis/test_subset_ps2/09'
#data preparation is different
#if using old scripts from xcdiff/original dpyscf,
#reference density matrices must be in ref_path/{INDEX}.dm.npy
#can just symlink the DMs produced by run_pyscf.py
#ADDITIONALLY, reference energies must be stored in ref_path/energies
#formatted as:
#IDX ENERGY
# mats2_o3 = get_datapoint(o3, basis='6-311++G(3df,2pd)', grid_level= 3,
#                                 xc='scan', zsym=o3.info.get('sym', False),
#                                 n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
#                                 init_guess=False, spin = 0,
#                                 pol=True, do_fcenter=True,
#                                 ref_path=ref_path, ref_index= 24,
#                                 ref_basis='6-311++G(3df,2pd)')
mats2_o = get_datapoint(o, basis='6-311++G(3df,2pd)', grid_level= 5,
                                xc='scan', zsym=o.info.get('sym', True),
                                n_rad=o.info.get('n_rad',30), n_ang=o.info.get('n_ang',15),
                                init_guess=True, spin = o.info.get('spin', 0),
                                pol=o.info.get('pol', True), do_fcenter=True,
                                ref_path=ref_path, ref_index= 20,
                                ref_basis='6-311++G(3df,2pd)')
#unpack returned tuples
# e_base2_o3, eye2_o3, mats2_o3 = mats2_o3
e_base2_o, eye2_o, mats2_o = mats2_o

Atoms(symbols='O', pbc=False)
6-311++G(3df,2pd)
Spin specified in atom info.
Single Atom -- UKS
Building grids...
METHOD GENERATED: <function UKS at 0x7f898b506170>


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/pyscf/gto/moleintor.py:259: UserWarning: Function int4c1e_sph not found.  Set its comp to 1
  warnings.warn('Function %s not found.  Set its comp to 1' % intor_name)


converged SCF energy = -75.0641194823042  <S^2> = 2.0054503  2S+1 = 3.0036313
Loading reference density


In [7]:
#need to generate more matrices for network -- this is usually handled by the data reader class used in training
# s_chol_o3 = np.linalg.inv(np.linalg.cholesky(mats2_o3['s']))
s_chol_o = np.linalg.inv(np.linalg.cholesky(mats2_o['s']))

#old version expects this under different key names
# mats2_o3['s_chol'] = s_chol_o3
# mats2_o3['s_oh'] = s_chol_o3
# mats2_o3['s_inv_oh'] = s_chol_o3.T

mats2_o['s_chol'] = s_chol_o
mats2_o['s_oh'] = s_chol_o
mats2_o['s_inv_oh'] = s_chol_o.T

#go through and make everything a list
# mats2_o3_l = {k: [mats2_o3[k]] for k in mats2_o3.keys()}
mats2_o_l = {k: [mats2_o[k]] for k in mats2_o.keys()}

#go through and additionally make everything a tensor
for k in mats2_o_l.keys():
    print(k)
#     o3k = mats2_o3_l[k]
    ok = mats2_o_l[k]
    if type(ok[0]) in [int, float]:
        print('integer/float from o3 dict')
#         mats2_o3_l[k] = [torch.Tensor(np.array(o3k))]
        mats2_o_l[k] = [torch.Tensor(np.array(ok))]
    elif not ok[0].shape:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0].reshape(1,))]
        mats2_o_l[k] = [torch.Tensor(ok[0].reshape(1,))]
    else:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0])]
        mats2_o_l[k] = [torch.Tensor(ok[0])]

dm_init
v
t
s
eri
n_elec
n_atoms
e_nuc
mo_energy
mo_occ
e_ip
ip_idx
e_pretrained
L
scaling
rho
ao_eval
grid_weights
fcenter
s_chol
s_oh
s_inv_oh


In [8]:
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA'
scf2 = lite_get_scf('MGGA', path=modelpath+'/xc')

sc=True
vvv=True

# res2_o3 = scf2([torch.Tensor(mats2_o3_l['dm_init'][0])], mats2_o3_l, sc=sc, verbose=vvv)
# res2_o3 = res2_o3['E']

res2_o = scf2([torch.Tensor(mats2_o_l['dm_init'][0])], mats2_o_l, sc=sc, verbose=vvv)
res2_o = res2_o['E']

# print('Dpyscf (XCDiff) Evaluated on:\nO3 = {}'.format(res2_o3))
print('Dpyscf (XCDiff) Evaluated on:\nO = {}'.format(res2_o))

FREEC False
SCF Loop Beginning: 25 Steps
Step 0


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)


0 Energy: tensor([-74.8719], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-43.0740], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0740], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-34.1759], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0740], grad_fn=<AddBackward0>), tensor([-34.1759], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-29.3460], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0740], grad_fn=<AddBackward0>), tensor([-34.1759], grad_fn=<AddBackward0>), tensor([-29.3460], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-26.1582], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0740], grad_fn=<AddBackward0>), tensor([-34.1759], grad_fn=<AddBackward0>), tensor([-29.3460], grad_fn=<AddBackward0>), tensor([-26.1582], grad_fn=<AddBackward0>

KeyboardInterrupt: 

# Dpyscf (original, contained in standalone repo folder, hosted here())

**WARNING: On my machine at least, PyTorch *does not release the memory used during an SCF cycle.* Doing the above calculations with 25 SCF steps uses ~50 GB of RAM, which might not get released until restarting this notebook's kernel. User beware.**

In [2]:
#for original+xcdiff dpyscf
from ogdpyscf.utils import get_datapoint as og_get_datapoint
from ogdpyscf.net import lite_get_scf

In [7]:
ref_path = '/home/awills/Documents/Research/swxcd/aegis/test_subset_ps2/09'
#data preparation is different
#if using old scripts from xcdiff/original dpyscf,
#reference density matrices must be in ref_path/{INDEX}.dm.npy
#can just symlink the DMs produced by run_pyscf.py
#ADDITIONALLY, reference energies must be stored in ref_path/energies
#formatted as:
#IDX ENERGY
# mats2_o3 = get_datapoint(o3, basis='6-311++G(3df,2pd)', grid_level= 3,
#                                 xc='scan', zsym=o3.info.get('sym', False),
#                                 n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
#                                 init_guess=False, spin = 0,
#                                 pol=True, do_fcenter=True,
#                                 ref_path=ref_path, ref_index= 24,
#                                 ref_basis='6-311++G(3df,2pd)')
mats2_o = og_get_datapoint(o, basis='6-311++G(3df,2pd)', grid_level= 5,
                                xc='scan', zsym=o.info.get('sym', True),
                                n_rad=o.info.get('n_rad',30), n_ang=o.info.get('n_ang',15),
                                init_guess=True, spin = o.info.get('spin', 0),
                                pol=o.info.get('pol', True), do_fcenter=True,
                                ref_path=ref_path, ref_index= 20,
                                ref_basis='6-311++G(3df,2pd)')
#unpack returned tuples
# e_base2_o3, eye2_o3, mats2_o3 = mats2_o3
e_base2_o, eye2_o, mats2_o = mats2_o

Atoms(symbols='O', pbc=False)
6-311++G(3df,2pd)


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/pyscf/gto/moleintor.py:259: UserWarning: Function int4c1e_sph not found.  Set its comp to 1
  warnings.warn('Function %s not found.  Set its comp to 1' % intor_name)


SCF not converged.
SCF energy = -75.0641922335818 after 50 cycles  <S^2> = 2.0054791  2S+1 = 3.0036505
Loading reference density
SCF not converged.
SCF energy = -75.0642126803569 after 50 cycles  <S^2> = 2.0054618  2S+1 = 3.003639


In [8]:
#need to generate more matrices for network -- this is usually handled by the data reader class used in training
# s_chol_o3 = np.linalg.inv(np.linalg.cholesky(mats2_o3['s']))
s_chol_o = np.linalg.inv(np.linalg.cholesky(mats2_o['s']))

#old version expects this under different key names
# mats2_o3['s_chol'] = s_chol_o3
# mats2_o3['s_oh'] = s_chol_o3
# mats2_o3['s_inv_oh'] = s_chol_o3.T

mats2_o['s_chol'] = s_chol_o
mats2_o['s_oh'] = s_chol_o
mats2_o['s_inv_oh'] = s_chol_o.T

#go through and make everything a list
# mats2_o3_l = {k: [mats2_o3[k]] for k in mats2_o3.keys()}
mats2_o_l = {k: [mats2_o[k]] for k in mats2_o.keys()}

#go through and additionally make everything a tensor
for k in mats2_o_l.keys():
    print(k)
#     o3k = mats2_o3_l[k]
    ok = mats2_o_l[k]
    if type(ok[0]) in [int, float]:
        print('integer/float from o3 dict')
#         mats2_o3_l[k] = [torch.Tensor(np.array(o3k))]
        mats2_o_l[k] = [torch.Tensor(np.array(ok))]
    elif not ok[0].shape:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0].reshape(1,))]
        mats2_o_l[k] = [torch.Tensor(ok[0].reshape(1,))]
    else:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0])]
        mats2_o_l[k] = [torch.Tensor(ok[0])]

dm_init
v
t
s
n_elec
n_atoms
e_nuc
mo_energy
mo_occ
e_ip
ip_idx
e_pretrained
eri
L
scaling
rho
ao_eval
grid_weights
fcenter
s_chol
s_oh
s_inv_oh


In [9]:
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA'
scf3 = lite_get_scf('MGGA', path=modelpath+'/xc')

sc=True
vvv=True

# res2_o3 = scf3([torch.Tensor(mats2_o3_l['dm_init'][0])], mats2_o3_l, sc=sc, verbose=vvv)
# res2_o3 = res2_o3['E']

res2_o = scf3([torch.Tensor(mats2_o_l['dm_init'][0])], mats2_o_l, sc=sc, verbose=vvv)
res2_o = res2_o['E']

# print('Dpyscf (XCDiff) Evaluated on:\nO3 = {}'.format(res2_o3))
print('Dpyscf (XCDiff) Evaluated on:\nO = {}'.format(res2_o))

FREEC False
SCF Loop Beginning: 25 Steps
Step 0


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)


0 Energy: tensor([-74.8719], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-43.0739], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0739], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-34.1753], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0739], grad_fn=<AddBackward0>), tensor([-34.1753], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-29.3467], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0739], grad_fn=<AddBackward0>), tensor([-34.1753], grad_fn=<AddBackward0>), tensor([-29.3467], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-26.1592], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0739], grad_fn=<AddBackward0>), tensor([-34.1753], grad_fn=<AddBackward0>), tensor([-29.3467], grad_fn=<AddBackward0>), tensor([-26.1592], grad_fn=<AddBackward0>

18 Energy: tensor([-18.1672], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0739], grad_fn=<AddBackward0>), tensor([-34.1753], grad_fn=<AddBackward0>), tensor([-29.3467], grad_fn=<AddBackward0>), tensor([-26.1592], grad_fn=<AddBackward0>), tensor([-23.9386], grad_fn=<AddBackward0>), tensor([-22.3661], grad_fn=<AddBackward0>), tensor([-21.2429], grad_fn=<AddBackward0>), tensor([-20.4249], grad_fn=<AddBackward0>), tensor([-19.8222], grad_fn=<AddBackward0>), tensor([-19.3677], grad_fn=<AddBackward0>), tensor([-19.0306], grad_fn=<AddBackward0>), tensor([-18.7566], grad_fn=<AddBackward0>), tensor([-18.5714], grad_fn=<AddBackward0>), tensor([-18.4530], grad_fn=<AddBackward0>), tensor([-18.3738], grad_fn=<AddBackward0>), tensor([-18.3338], grad_fn=<AddBackward0>), tensor([-18.2365], grad_fn=<AddBackward0>), tensor([-18.1672], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-18.1286], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<

# Pasted Outputs

## Dpyscfl Output

In case the output gets erased for some reason and the above cell can't be run, the output is:
```
FREEC False
SCF Loop Beginning: 25 Steps
Step 0
Noise generation to avoid potential degeneracies
0 Energy: tensor([-225.1551], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-225.1555], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-225.1556], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>)]
Step 5
5 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 17

17 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 18
18 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>)]
SCF Loop Beginning: 25 Steps
Step 0

Noise generation to avoid potential degeneracies
0 Energy: tensor([-74.9761], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-74.9764], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-74.9764], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 5
5 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 17
17 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 18

18 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Dpyscf-lite (XCDiff) Evaluated on:
O3 = tensor([-225.1551, -225.1555, -225.1556, -225.1557, -225.1557, -225.1558,
        -225.1558, -225.1558, -225.1558, -225.1558, -225.1558, -225.1558,
        -225.1558, -225.1558, -225.1558, -225.1558, -225.1558, -225.1558,
        -225.1558, -225.1558, -225.1558, -225.1558, -225.1558, -225.1558,
        -225.1557], grad_fn=<CatBackward0>)
O = tensor([-74.9761, -74.9764, -74.9764, -74.9765, -74.9765, -74.9765, -74.9765,
        -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765,
        -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765,
        -74.9765, -74.9765, -74.9765, -74.9765], grad_fn=<CatBackward0>)

```

## Original Dpyscf Output

In case the output gets erased for some reason and the above cell can't be run, the output is:
```
FREEC False

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)

SCF Loop Beginning: 25 Steps
Step 0
0 Energy: tensor([-225.1550], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-118.4661], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-89.9705], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-76.8356], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-68.9395], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>)]
Step 5
5 Energy: tensor([-63.6487], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-60.0146], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-57.4984], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-55.7549], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-54.5512], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-53.7476], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-53.2192], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-52.8860], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-52.6544], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-52.5047], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-52.4044], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-52.3478], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>)]
Step 17
17 Energy: tensor([-52.3053], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>)]
Step 18

18 Energy: tensor([-52.2767], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-52.2593], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-52.2522], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-52.2416], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-52.2417], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>), tensor([-52.2417], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-52.2341], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>), tensor([-52.2417], grad_fn=<AddBackward0>), tensor([-52.2341], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-52.2365], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>), tensor([-52.2417], grad_fn=<AddBackward0>), tensor([-52.2341], grad_fn=<AddBackward0>), tensor([-52.2365], grad_fn=<AddBackward0>)]
SCF Loop Beginning: 25 Steps
Step 0
0 Energy: tensor([-74.9761], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-43.2015], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-34.2806], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-29.4604], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-26.3482], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>)]
Step 5

5 Energy: tensor([-24.1357], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-22.5100], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-21.3647], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-20.5265], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-19.8936], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-19.4032], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-19.0713], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-18.8011], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-18.6086], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-18.4598], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-18.3561], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-18.2716], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>)]
Step 17
17 Energy: tensor([-18.1856], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>)]
Step 18
18 Energy: tensor([-18.0978], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>)]
Step 19

19 Energy: tensor([-18.0366], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-18.0095], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-17.9769], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-17.9236], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>), tensor([-17.9236], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-17.8774], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>), tensor([-17.9236], grad_fn=<AddBackward0>), tensor([-17.8774], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-17.8375], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>), tensor([-17.9236], grad_fn=<AddBackward0>), tensor([-17.8774], grad_fn=<AddBackward0>), tensor([-17.8375], grad_fn=<AddBackward0>)]
Dpyscf (XCDiff) Evaluated on:
O3 = tensor([-225.1550, -118.4661,  -89.9705,  -76.8356,  -68.9395,  -63.6487,
         -60.0146,  -57.4984,  -55.7549,  -54.5512,  -53.7476,  -53.2192,
         -52.8860,  -52.6544,  -52.5047,  -52.4044,  -52.3478,  -52.3053,
         -52.2767,  -52.2593,  -52.2522,  -52.2416,  -52.2417,  -52.2341,
         -52.2365], grad_fn=<CatBackward0>)
O = tensor([-74.9761, -43.2015, -34.2806, -29.4604, -26.3482, -24.1357, -22.5100,
        -21.3647, -20.5265, -19.8936, -19.4032, -19.0713, -18.8011, -18.6086,
        -18.4598, -18.3561, -18.2716, -18.1856, -18.0978, -18.0366, -18.0095,
        -17.9769, -17.9236, -17.8774, -17.8375], grad_fn=<CatBackward0>)
```